In [164]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [165]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#pd.set_option('display.max_rows', None)

 train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
* PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
* HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
* CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
* Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
* Destination - The planet the passenger will be debarking to.
* Age - The age of the passenger.
* VIP - Whether the passenger has paid for special VIP service during the voyage.
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
* Name - The first and last names of the passenger.
* Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.
* test.csv - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of * Transported for the passengers in this set.

sample_submission.csv - A submission file in the correct format.

* PassengerId - Id for each passenger in the test set.
* Transported - The target. For each passenger, predict either True or False.

In [166]:
#test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test = pd.read_csv('test.csv')


# Separate test PassengerId (will need it for submission)
test_pass_id = test.PassengerId.copy()

In [167]:
#train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train = pd.read_csv('train.csv')


train_pass_id = train.PassengerId.copy()

In [168]:
X = train.drop(columns = 'Transported')
y = train[['Transported']]

In [169]:
# vamos unir o DF de test e de treino para obter melhores análises
df = pd.concat([X, test], ignore_index = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
dtypes: float64(6), object(7)
memory usage: 1.3+ MB


In [170]:
nan_inicial = df.isna().sum()
nan_inicial

PassengerId       0
HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Name            294
dtype: int64

In [171]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [172]:
df[['deck', 'num', 'side']] = df['Cabin'].str.split(pat = '/', expand = True)
df[['Passenger', '_Id']] = df['PassengerId'].str.split(pat = '_', expand = True)
df[['Nome', 'Sobrenome']] = df['Name'].str.split(pat = ' ', expand = True)
df.drop(columns = ['Cabin', 'Name'], inplace = True)
df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,Passenger,_Id,Nome,Sobrenome
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0,P,0001,01,Maham,Ofracculy
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0,S,0002,01,Juanna,Vines
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S,0003,01,Altark,Susent
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S,0003,02,Solam,Susent
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1,S,0004,01,Willy,Santantines


In [173]:
df.HomePlanet.unique()

array(['Europa', 'Earth', 'Mars', nan], dtype=object)

In [174]:
df.replace({ False : 0,
                 True: 1}, inplace = True)

In [175]:
#fig, axes = plt.subplots(2, 3, sharey=True, figsize=(15, 7))
#df.plot.scatter(x='RoomService', y='Age', ax=axes[0, 0])
#df.plot.scatter(x='FoodCourt', y='Age', ax=axes[0, 1])
#df.plot.scatter(x='ShoppingMall', y='Age', ax=axes[0, 2])
#df.plot.scatter(x='Spa', y='Age', ax=axes[1, 0])
#df.plot.scatter(x='VRDeck', y='Age', ax=axes[1, 1])
#plt.show()

In [176]:
# Cap RoomService at 9000
#df.loc[df.RoomService.gt(8000), 'RoomService'] = 8000

# Cap FoodCourt at 22000
#df.loc[df.FoodCourt.gt(20000), 'FoodCourt'] = 20000

# Cap ShoppingMall at 11000
#df.loc[df.ShoppingMall.gt(10000), 'ShoppingMall'] = 10000

# Cap Spa at 17000
#df.loc[df.Spa.gt(16000), 'Spa'] = 16000

# Cap VRDeck at 21000
#df.loc[df.VRDeck.gt(20000), 'VRDeck'] = 20000

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.gt.html

In [177]:
# Vamos começar as análises por HomePlanet

df.groupby(['HomePlanet']).deck.value_counts() 

HomePlanet  deck
Earth       G       3700
            F       2426
            E        583
Europa      B       1124
            C       1081
            A        346
            D        296
            E        197
            T         10
Mars        F       1713
            E        508
            D        406
Name: deck, dtype: int64

- Decks *G* só transportam pessoas da terra
- Terra só possui decks *G*, *E*, *F*
- Deck *T* só sai de Europa
- Europa só possui decks *A*, *B*, *C*, *D*, *E* e *T*
- Europa só possui decks *D*, *E* e *F*

In [178]:
df.loc[df.deck == 'G', 'HomePlanet' ] = 'Earth'
df.loc[df.deck == 'T', 'HomePlanet' ] = 'Europa'

In [179]:

df.groupby(['VIP',]).deck.value_counts()


VIP  deck
0.0  F       4102
     G       3693
     E       1274
     B       1058
     C       1018
     D        666
     A        299
     T         10
1.0  B         58
     C         55
     A         45
     D         41
     F         41
     E         26
Name: deck, dtype: int64

- Só temos VIPs nos terminais A, B, C, D, E e F
- Terminais G e T não possuem vips

In [180]:
df.loc[(df.deck == 'G') | (df.deck == 'T'), 'VIP' ] = 0

In [181]:
df.groupby(['CryoSleep','Destination']).deck.value_counts()

CryoSleep  Destination    deck
0.0        55 Cancri e    F        458
                          G        230
                          C        222
                          B        202
                          E        120
                          D        107
                          A        103
                          T          2
           PSO J318.5-22  F        336
                          G        161
                          E         44
                          D         17
                          C          8
                          B          3
                          A          1
           TRAPPIST-1e    F       2423
                          G       1236
                          E        847
                          D        402
                          C        368
                          B        291
                          A        135
                          T          8
1.0        55 Cancri e    B        293
                          G      

- Deck T não transporta  CryoSleep
- A qtd de pessoas que saem com CryoSleep da terra nos decks E e F são mínimas
- Mars -> PSO, Deck D somente SEM Cryo

In [182]:
df.loc[(df.deck == 'T') , 'CryoSleep' ] = 0
df.loc[(df.HomePlanet == 'Mars') & (df.Destination == 'PSO J318.5-22') & (df.deck == 'D') , 'CryoSleep' ] = 0

In [183]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,Passenger,_Id,Nome,Sobrenome
0,0001_01,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,B,0,P,0001,01,Maham,Ofracculy
1,0002_01,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,F,0,S,0002,01,Juanna,Vines
2,0003_01,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,A,0,S,0003,01,Altark,Susent
3,0003_02,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,A,0,S,0003,02,Solam,Susent
4,0004_01,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,F,1,S,0004,01,Willy,Santantines


In [184]:
#Avaliar os gastos das pessoas por VIP e CyooSleep
df.groupby(['CryoSleep','VIP']).agg({'RoomService' : ['mean', 'min', 'max'],
                                    'FoodCourt' : ['mean', 'min', 'max'],
                                    'ShoppingMall' : ['mean', 'min', 'max'],
                                    'Spa' : ['mean', 'min', 'max'],
                                    'VRDeck' : ['mean', 'min', 'max']})

RoomService                  FoodCourt                \
                     mean  min      max         mean  min      max   
CryoSleep VIP                                                        
0.0       0.0  342.545455  0.0  14327.0   660.739772  0.0  27071.0   
          1.0  512.165957  0.0   8030.0  2015.370213  0.0  29813.0   
1.0       0.0    0.000000  0.0      0.0     0.000000  0.0      0.0   
          1.0    0.000000  0.0      0.0     0.000000  0.0      0.0   

              ShoppingMall                        Spa                \
                      mean  min      max         mean  min      max   
CryoSleep VIP                                                         
0.0       0.0   274.284710  0.0  23492.0   461.192246  0.0  22408.0   
          1.0   308.742489  0.0   3700.0  1040.220339  0.0  15255.0   
1.0       0.0     0.000000  0.0      0.0     0.000000  0.0      0.0   
          1.0     0.000000  0.0      0.0     0.000000  0.0      0.0   

                    VRDeck                
                      mean  min      max  
CryoSleep VIP                             
0.0       0.0   451.563769  0.0  24133.0  
          1.0  1319.000000  0.0  19086.0  
1.0       0.0     0.000000  0.0      0.0  
          1.0     0.000000  0.0      0.0

Todos que estão em CRIO possuem gastos zero

In [185]:
df.loc[(df.CryoSleep == 1) , ['RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck' ] ] = 0

In [186]:
nan_df = pd.DataFrame()
nan_df['inicial'] = nan_inicial
nan_df['atual'] = df.isna().sum()
nan_df['dif'] = nan_df['inicial'] - nan_df['atual']
nan_df


,inicial,atual,dif
PassengerId,0,0.0,0.0
HomePlanet,288,206.0,82.0
CryoSleep,310,309.0,1.0
Cabin,299,NaN,NaN
Destination,274,274.0,0.0
Age,270,270.0,0.0
VIP,296,207.0,89.0
RoomService,263,170.0,93.0
FoodCourt,289,180.0,109.0
ShoppingMall,306,175.0,131.0


In [189]:
#Passageiro do mesmo grupo são sempre do mesmo HomePlanet
lista = df.groupby(['Passenger']).count().sort_values('_Id', ascending = False)
lista

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,_Id,Nome,Sobrenome
Passenger,,,,,,,,,,,,,,,,,
4005,8,8,8,8,8,8,8,8,8,8,8,7,7,7,8,7,7
0984,8,8,8,8,7,8,8,8,8,8,8,8,8,8,8,7,7
8728,8,8,7,8,8,8,8,8,8,8,8,7,7,7,8,8,8
6332,8,8,6,8,8,8,7,8,8,8,8,8,8,8,8,8,8
5756,8,8,7,8,8,8,8,8,8,8,8,7,7,7,8,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3503,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3502,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3501,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1


In [194]:
lista = lista.loc[lista.HomePlanet > 1].index.tolist()


for i in lista:
    df.loc[(df.Passenger == i),'HomePlanet' ] = df.loc[(df.Passenger == i),'HomePlanet'].value_counts().idxmax()

In [ ]:
df.groupby(['num', 'Sobrenome']).count().sort_values('_Id', ascending = False).head(10)
#Tenho uma terioria de que passageiros ficam alinhados por Passanger DEVEM possuir os mesmos dados

In [195]:
df.isna().sum()

PassengerId       0
HomePlanet      153
CryoSleep       309
Destination     274
Age             270
VIP             207
RoomService     170
FoodCourt       180
ShoppingMall    175
Spa             177
VRDeck          177
deck            299
num             299
side            299
Passenger         0
_Id               0
Nome            294
Sobrenome       294
dtype: int64

In [ ]:
#df.fillna(method = 'ffill', inplace = True)
#pesquisar sobre o fillna com groupby 

JOGANDO O NOSSO DF ATUALIZADO PARA DENTRO DO ML

In [ ]:
X_novo = df.loc[df.PassengerId.isin(train_pass_id)]
X_novo.head()

In [ ]:
X_novo['num'] = pd.to_numeric(X_novo['num'])
X_novo['_Id'] = pd.to_numeric(X_novo['_Id'])
X_novo.drop(columns = ['PassengerId', 'Passenger', '_Id', 'Nome', 'Sobrenome'], inplace = True)

In [ ]:
# separando os valores de cabine
X[['deck', 'num', 'side']] = X['Cabin'].str.split(pat = '/', expand = True)
X.drop(columns = 'Cabin', inplace = True)


#substiutior os False and Trues por 0 e 1
X.replace({ False : 0,
                 True: 1}, inplace = True)

y.replace({ False : 0,
                 True: 1}, inplace = True)

# em uma primeira abordagem, vamos substituir todos os NaN pelos valores mais frequentes
# será ralizado dentro de um dataframe
X['num'] = pd.to_numeric(X['num'])

## Tive que remover as duas colunas abaixo pois estavam apresentando problemas no pipeline, depois vou investigar, mas acredito que deva ser por causa do inputer.
X.drop(columns = ['PassengerId', 'Name'], inplace = True)



# separando as colunas entre numéricas e categóricas
colunas_numericas = X.select_dtypes(include=np.number).columns
colunas_numericas

colunas_categoricas = X.select_dtypes(include = 'object').columns
colunas_categoricas


In [ ]:
X_novo.head()

In [ ]:
colunas_numericas = X_novo.select_dtypes(include=np.number).columns
colunas_numericas

colunas_categoricas = X_novo.select_dtypes(include = 'object').columns
colunas_categoricas


# Estrutura do pipeline

In [ ]:
# Preprocessamento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn import svm
import lightgbm as lgb



In [ ]:
#Split de treino e teste
SEED = 123
X_train, X_test, y_train, y_test = train_test_split(X_novo, y, test_size=0.3, random_state=SEED)

#preprocessor
numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='most_frequent'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='most_frequent'))
      ,('encoder', OneHotEncoder()) #depois testar com OrdinalEncoder
])



numeric_features = colunas_numericas 
categorical_features = colunas_categoricas

preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
]) 

In [ ]:
#Estruturando os pipelines


pipe_lr = Pipeline([('preprocessor', preprocessor),
                    ('LR', LogisticRegression(random_state = SEED))])
pipe_dt = Pipeline([('preprocessor', preprocessor),
                    ('DT',DecisionTreeClassifier(random_state = SEED))])
pipe_rf = Pipeline([('preprocessor', preprocessor),
                    ('RF',RandomForestClassifier(random_state = SEED))])
pipe_knn = Pipeline([('preprocessor', preprocessor),
                    ('KNN', KNeighborsClassifier())])
pipe_svm = Pipeline([('preprocessor', preprocessor),
                     ('SVM', svm.SVC(random_state = SEED))])
pipe_xgb = Pipeline([('preprocessor', preprocessor),
                     ('XGB', XGBClassifier(random_state = SEED))])
pipe_lgb = Pipeline([('preprocessor', preprocessor),
                     ('LGB', lgb.LGBMClassifier(random_state = SEED))])


In [ ]:
pipelines = [pipe_lr, pipe_dt, pipe_rf, pipe_knn, pipe_svm, pipe_xgb, pipe_lgb]
for pipe in pipelines:
  pipe.fit(X_train, y_train.values.ravel())

In [ ]:
grid_dict = {0: 'Logistic Regression', 1: 'Decision Trees', 
             2: 'Random Forest', 3: 'K-Nearest Neighbors', 
             4: 'Support Vector Machines', 5: 'XGBoost',
             6: 'LightGBM'}
for i, model in enumerate(pipelines):
    print('{} Test Accuracy: {}'.format(grid_dict[i], model.score(X_test,y_test)))
    #print('{} Best Params: {}'.format(grid_dict[i], model.best_params_))

In [ ]:
#Vamos tunar os algoritimos de LR, RF, SVM e XGBOOST

param_range = [1, 2, 3, 4, 5, 6]
param_range_fl = [1.0, 0.5, 0.1]
n_estimators = [50,100,150, 200, 250]
learning_rates = [.1,.2,.3]

lr_param_grid = [{'LR__penalty': ['l1', 'l2'],
                   'LR__C': param_range_fl,
                   'LR__solver': ['liblinear']}]

rf_param_grid = [{'RF__min_samples_leaf': param_range,
                   'RF__max_depth': param_range,
                   'RF__min_samples_split': param_range[1:]}]

#svm_param_grid = [{'SVM__kernel': ['linear', 'rbf'], 
#                    'SVM__C': [0.1, 1, 5, 10, 100, 1000],
#                    'SVM__gamma' : [1, 0.1, 0.01, 0.001, 0.0001 ]}]
svm_param_grid = [{'SVM__kernel': ['linear', 'rbf'], 
                    'SVM__C': param_range}]


xgb_param_grid = [{'XGB__learning_rate': learning_rates,
                    'XGB__max_depth': param_range,
                    'XGB__min_child_weight': param_range[:2],
                    'XGB__subsample': param_range_fl,
                    'XGB__n_estimators': n_estimators}]

#xgb_param_grid = [{'XGB__n_estimators': [100, 200, 500],
#                    'XGB__learning_rate': [0.01,0.05,0.1],
#                    'XGB__booster': ['gbtree', 'gblinear'],
#                    'XGB__gamma': [0, 0.5, 1],
#                    'XGB__reg_alpha': [0, 0.5, 1],
#                    'XGB__reg_lambda': [0.5, 1, 5],
#                    'XGB__base_score': [0.2, 0.5, 1]
#}]

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

#lgb_param_grid = [{'LGB__num_leaves': sp_randint.rvs(6, 50), 
#             'LGB__min_child_samples': sp_randint.rvs(100, 500), 
##             'LGB__min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#             'LGB__subsample': sp_uniform(loc=0.2, scale=0.8), 
#             'LGB__colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#             'LGB__reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#             'LGB__reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}]


In [ ]:
from sklearn.model_selection import GridSearchCV

lr_grid_search = GridSearchCV(estimator=pipe_lr,
        param_grid=lr_param_grid,
        scoring='accuracy',
        cv=3)

rf_grid_search = GridSearchCV(estimator=pipe_rf,
        param_grid=rf_param_grid,
        scoring='accuracy',

        cv=3)

svm_grid_search = GridSearchCV(estimator=pipe_svm,
        param_grid=svm_param_grid,
        scoring='accuracy',
        cv=3)

xgb_grid_search = GridSearchCV(estimator=pipe_xgb,
        param_grid=xgb_param_grid,
        scoring='accuracy',
        cv=3)

#lgb_grid_search = GridSearchCV(estimator=pipe_lgb,
#        param_grid=lgb_param_grid,
#        scoring='accuracy',
#        cv=3)


In [ ]:
a

In [ ]:
grids = [lr_grid_search, rf_grid_search, svm_grid_search, xgb_grid_search]
for pipe in grids:
    pipe.fit(X_train,y_train.values.ravel())

In [ ]:
grid_dict = {0: 'Logistic Regression',
             1: 'Random Forest', 
             2: 'Support Vector Machines', 3: 'XGBoost'}
for i, model in enumerate(grids):
    print('{} Test Accuracy: {}'.format(grid_dict[i], model.score(X_test,y_test)))
    #print('{} Best Params: {}'.format(grid_dict[i], model.best_params_))

In [ ]:
## Gerando o CSV de submissão:
# realizando as mudanças que fizemos no dataset original FORA do pipeline:

test = df.loc[df.PassengerId.isin(test_pass_id)]

# gerando o modelo

#modelo = pipe_lgb.fit(X_train, y_train)
modelo = xgb_grid_search.fit(X_train, y_train)

# gerando y_previsto
y_previsto = modelo.predict(test) 
y_previsto

In [ ]:
# Save predictions in the format used for competition scoring
#output = pd.DataFrame({'PassengerId': test_pass_id,
#                       'Transported': y_previsto.astype(bool)})
#
#output.to_csv('submission.csv', index=False)

Testando coisas que vi por ai na net